# **Generating all possible equations for [Nerdle](https://nerdlegame.com/) game** *([Wordle](https://www.nytimes.com/games/wordle/index.html)'s mathematic variant)*

Let's list the explicit rules for each possible equation:

+ Depending on the game mode, each expression must be exactly 6 or 8 elements long *(each element is considered to be a single digit number, an operator or the equal sign)*
+ Both the expression and solution parts of the equation can't start with any operator *(`+, -, /, *`)*
+ Numbers and operators can appear more than once *(except the equal sign)*
+ No leading zeros are allowed
+ Calculations follow the order of operations **PEDMAS**
    1. Parenthesis
    2. Exponents
    3. Division or Multiplication
    4. Addition or Subtraction

We can also extrapolate some other rules based on the previous requirements:

+ Both the expression and solution parts of the equation can't end with any operator *(`+, -, /, *`)*
+ There can't be expressions that are equal to the solution
+ Only expressions that evaluate to an integer are valid

Besides all of that, there are 2 game modes:

+ **Mini Nerdle** *(6 elements)*
+ **Nerdle** *(8 elements)*

In [1]:
from pprint import pprint
from tqdm import tqdm
import itertools
import csv
import re

## Defining constants

In [2]:
GAME_MODES = {
    "MINI_NERDLE": {
        "NUMBER_OF_ELEMENTS": 6,
        "NUMBER_OF_ATTEMPTS": 6,
    },
    "NERDLE": {
        "NUMBER_OF_ELEMENTS": 8,
        "NUMBER_OF_ATTEMPTS": 6,
    },
}


FILENAME_PREFIX = "equations"   # The prefix for the output files
CSV_HEADER = ["equation"]       # The header for the CSV files

## Function that generates all possible patterns for the equations

For "Regular Mode" The equal sign can only be in 3 valid positions:

+ Position 4: "....=..."
+ Position 5: ".....=.."
+ Position 6: "......=."

For "Mini Mode" The equal sign can only be in 2 valid positions:

+ Position 3: "...=.."
+ Position 4: "....=."

There is no point in generating equations that have the equal sign somewhere else, so we will create only the patterns that fit this constraint.

In [3]:
def gen_patterns(game_mode: str):

    # Extract variables
    number_of_elements = GAME_MODES[game_mode]["NUMBER_OF_ELEMENTS"]

    # Init patterns list
    patterns = []
    
    # Iterating over the equal sign's valid positions
    for equal_sign_idx in range(4, int((number_of_elements / 2) + 3)):

        # Init pattern
        pattern = []

        # Append the first expression position elements (There can't be any operators or zeros in here)
        pattern.append(['1', '2', '3', '4', '5', '6', '7', '8', '9'])

        # Append the "middle" expression positions elements (All numbers and operators are valid)
        for _ in range(equal_sign_idx + int(((number_of_elements - 8) / 2) - 2)):
            pattern.append(['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '+', '-', '*', '/'])
        
        # Append the "last" expression position elements (No operators are allowed here)
        pattern.append(['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'])

        # Append the "=" sign
        pattern.append(["="])

        # Check how many elements are left for the result part of the equation
        if equal_sign_idx < (number_of_elements / 2) + 2:

            # Append the first result position elements (There can't be any zeros in here)
            pattern.append(['1', '2', '3', '4', '5', '6', '7', '8', '9'])

            # Append the "middle" and "last" result positions elements (All numbers are valid)
            for _ in range(equal_sign_idx + 2, int((number_of_elements / 2) + 4)):
                pattern.append(['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'])
        
        # If there is only one element in the result, append all numbers to it
        else:
            pattern.append(['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'])

        # Append generated pattern to the patterns list
        patterns.append(pattern)

    # Return all gererated patterns
    return patterns

#### Generating and previewing the patterns

In [4]:
game_mode = "MINI_NERDLE"

GAME_MODES[game_mode]["patterns"] = gen_patterns(game_mode)

# Preview
for pattern in GAME_MODES[game_mode]["patterns"]:
    pprint(pattern)
    print()

[['1', '2', '3', '4', '5', '6', '7', '8', '9'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '+', '-', '*', '/'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'],
 ['='],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']]

[['1', '2', '3', '4', '5', '6', '7', '8', '9'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '+', '-', '*', '/'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '+', '-', '*', '/'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'],
 ['='],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']]



In [5]:
game_mode = "NERDLE"

GAME_MODES[game_mode]["patterns"] = gen_patterns(game_mode)

# Preview
for pattern in GAME_MODES[game_mode]["patterns"]:
    pprint(pattern)
    print()

[['1', '2', '3', '4', '5', '6', '7', '8', '9'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '+', '-', '*', '/'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '+', '-', '*', '/'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'],
 ['='],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']]

[['1', '2', '3', '4', '5', '6', '7', '8', '9'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '+', '-', '*', '/'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '+', '-', '*', '/'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '+', '-', '*', '/'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0'],
 ['='],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']]

[['1', '2', '3', '4', '5', '6', '7', '8', '9'],
 ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '+', '-', '*', '/'],
 ['1', '2', '3', '4', '5', '6', '7', '8

## Function that validates and solves an expression

In [6]:
def solve_expression(expression: str):
    
    ##################################################
    # PRE VALIDATION
    ##################################################
    
    # Discard consecutive operators, leading zeros and lone zeros
    if re.findall(r"[+,\-,*,\/]{2}|^0+|\D0+", expression):
        return None

    ##################################################
    # JOINING NUMBERS
    ##################################################

    # Convert expression to a list
    expression = list(expression)

    # Init temporary string and index number
    temp = ""
    i = 0

    # Loop through the expression
    while i < len(expression):

        # Check if the current element is a number
        if expression[i].isnumeric():

            # Add the number to the temporary string and pop it
            temp += expression[i]
            expression.pop(i)

        # Insert temp into expression if it's not empty
        else:
            if temp != "":
                expression.insert(i, temp)

            # Reset temp and go to the next element
            temp = ""
            i += 1

    # Append the last temp to the expression
    expression.append(temp)

    ##################################################
    # JOINING NUMBERS TO OPERATORS
    ##################################################

    # Init index
    i = 1

    # Loop through the expression while there are "+" and/or "-" operators
    while "+" in expression or "-" in expression:

        # Check if previous element is a "+" or "-" operator
        if expression[i - 1] in "+-":

            # If so, join the operator to the current number element, and pop current element
            expression[i - 1] += expression[i]
            expression.pop(i)

        # Go to next element
        i += 1

    ##################################################
    # CONVERTING NUMBERS TO INTS
    ##################################################

    # Loop through the expression
    for i in range(len(expression)):

        # Try to convert the element to an integer
        try:
            expression[i] = int(expression[i])
        except ValueError:
            pass

    ##################################################
    # SOLVING MULTIPLICATIONS AND DIVISIONS
    ##################################################

    # Init index
    i = 1

    # Loop through the expression while there are "/" and/or "*" operators
    while "/" in expression or "*" in expression:

        # Check if current element is a division or multiplication operator
        if expression[i] == "/" or expression[i] == "*":

            # Perform the appropriate operation
            if expression[i] == "/":
                expression[i - 1] /= expression[i + 1]

                # # Return None if the division results is not an integer
                # if not expression[i - 1] % 1 == 0:
                #     return None

            else:
                expression[i - 1] *= expression[i + 1]

            # Remove the current element and the following element
            for _ in range(2):
                expression.pop(i)

        # Go to next element
        else:
            i += 1

    ##################################################
    # SUMMING ALL ELEMENTS
    ##################################################

    # Return sum only if expression is not empty
    if expression:
        result = sum(expression)

        if result % 1 == 0:
            return int(result)

Testing the expressions solver

In [7]:
# Define test cases
TEST_EQUATIONS = [
    ("-3+4/2", -1),
    ("8+6-14", 0),
    ("-10+10", 0),
    ("-10+10", 0),
    ("10-1-9", 0),
    ("40/4/2", 5),
    ("1+2+30", 33),
    ("3+21/3", 10),
    ("40/4+2", 12),
    ("16+4*2", 24),
    ("2/1*40", 80),
    ("195-87", 108),
    ("801/89", 9),
    # Divisions not resulting in integers
    ("8/10*5", 4),
    ("8/12*9", 6),
    # Divisions by zero
    ("1+2/00", None),
    ("2/0*10", None),
    # Lone zeros
    ("0+50/1", None),
    ("50/1+0", None),
    ("50-0+2", None),
    ("4+00-3", None),
    # Leading zeros
    ("05-1+2", None),
    ("9+04/2", None),
    ("6/3+09", None),
]

# Iterate over equations and check if they match the result
for expression, output_expected in TEST_EQUATIONS:
    output_solver = solve_expression(expression)

    assert output_solver == output_expected, f"{expression} should return {output_expected} not {output_solver}"
else:
    print("All tests passed!")

All tests passed!


## Function that validates a result

In [8]:
def solve_result(result):

    # Discard leading zeros
    if re.findall(r"^0+[1-9]", result):
        return None
    
    # Return the result as an integer
    return int(result)

Testing the results solver

In [9]:
# Define test cases
TEST_RESULTS = [
    ("0", 0),
    ("9", 9),
    ("10", 10),
    ("78", 78),
    ("234", 234),
    ("908", 908),
    ("500", 500),
    # Leading zeros
    ("05", None),
    ("006", None),
    ("035", None),
]

# Iterate over results and check if they match the result
for result, output_expected in TEST_RESULTS:
    output_validator = solve_result(result)

    assert output_validator == output_expected, f"{result} should return {output_expected} not {output_validator}"
else:
    print("All tests passed!")

All tests passed!


## Function that generates all possible equations based on passed patterns

In [10]:
def gen_equations(game_mode: str):

    # Extracting variables
    patterns = GAME_MODES[game_mode]["patterns"]

    # Define variables
    output_filepath = f"./data/0.raw/{FILENAME_PREFIX}_{game_mode.lower()}.csv"
    total_expressions_counter = 0
    valid_expressions_counter = 0
    
    # Init output file
    with open(output_filepath, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(CSV_HEADER)

    # Iterate over all patterns
    for i, pattern in enumerate(patterns):

        # Calculate total number of candidate equations for a given pattern
        candidates_count = 1
        for subpattern in pattern:
            candidates_count *= len(subpattern)

        # Create all combinations of elements for each position of the equations pattern
        equations = itertools.product(*pattern)

        for equation in tqdm(equations, total=candidates_count, desc=f"pattern {i}\t"):

            # Convert equation into a single string
            equation = "".join(equation)

            # Split the equation into expression and result
            expression, result = equation.split("=")

            # Solve and validate both the expression and result 
            solved_result = solve_expression(expression)
            generated_result = solve_result(result)

            # Check if equation is valid and
            if solved_result is not None and generated_result is not None and solved_result == generated_result:

                # Append equation to the output file
                with open(output_filepath, "a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow([equation])

                # Increment valid expressions counter
                valid_expressions_counter += 1
            
            total_expressions_counter += 1

    # Print the valid number of expressions generated
    print(f"{valid_expressions_counter:,} valid expressions generated out of {total_expressions_counter:,} total expressions - ({valid_expressions_counter / total_expressions_counter * 100:.3f}%)")

#### Generating all possible candidate equations that follow the specified patterns

In [11]:
gen_equations(game_mode="MINI_NERDLE")

pattern 1	: 100%|██████████| 176400/176400 [00:01<00:00, 165381.85it/s]

206 valid expressions generated out of 289,800 total expressions - (0.071%)


In [12]:
gen_equations(game_mode="NERDLE")

pattern 2	: 100%|██████████| 34574400/34574400 [03:58<00:00, 144930.51it/s]

17,723 valid expressions generated out of 72,676,800 total expressions - (0.024%)
